In [ ]:
`!pip install altair vega_datasets

# Destilacion de Logits

In [59]:
import pandas as pd
import altair as alt
import lib.utils.viz.KD as KD
folder="./Cifar10/ResNet101/exp1/students"

In [60]:
folder="./Cifar10/ResNet101/exp8/students"
def acc_plot(data,field="acc",center=False,shape=(400,400),scale=None):
    detalle=['test_acc', 'test_teacher/acc', 'test_loss', 'test_eval',
           'train_acc', 'train_teacher/acc', 'train_loss', 'train_eval','distillation', 'temp']
    
    field_dict={'acc':"Accuracy [%]", 'eval':"Perdida Cross Entropy", 'loss':"Perdida de Destilación"
        
    }
    
    if scale is None:
        scale='log' if field in ['loss','eval'] else 'linear'
    else:
        print("scale", scale)
    
    print("lasorra")
    #data['train_acc']-=data['test_acc']
    
    
    bar=alt.Chart(data).mark_point().encode(
        alt.X('train_acc', scale=alt.Scale(zero=False,base=10,type=scale, ),title="Accuracy en entrenamiento"),
        alt.Y('test_acc', scale=alt.Scale(zero=False,base=10,type=scale, ),title="Accuracy en test"),
        shape=alt.Shape('distillation', legend=alt.Legend(title="Destilación")),
        color=alt.Color('student', legend=alt.Legend(title="Modelo")),
        size=alt.value(50),
        tooltip=detalle
        ).interactive()
    
    accs = {'Model':['MobileNet','ResNet18','ResNet101'],
                  'ce_train':[95.73,98.15,98.52],
                  'ce_test':[87.8,90.58,90.68]}
    if center:
        for phase in ["test","train"]:
            d=dict(list(zip(accs['Model'],accs['ce_%s'%phase])))
            data['%s_acc'%phase]-=[d[i] for i in data['student']]        
    else:

        df=pd.DataFrame(accs)

        aggregatex = alt.Chart(df).mark_rule(opacity=0.5).encode(
                    x='ce_train',
                    color='Model:N',
                    size=alt.value(2))

        aggregatey = alt.Chart(df).mark_rule(opacity=0.5).encode(
                    y='ce_test',
                    color='Model:N',
                    size=alt.value(2))

        return (aggregatey+aggregatex+bar).properties(width=shape[0],height=shape[1]) 
    return bar.properties(width=shape[0],height=shape[1])
        
data=KD.load_data(folder=folder)
acc_plot(data,scale="linear")
        

scale linear
lasorra


alt.LayerChart(...)

In [62]:

KD.load_and_plot(folder,phase='test',field="acc",shape=(400,400))#, center=True)

.
lasorra


alt.LayerChart(...)

In [34]:
data=KD.load_data(folder=folder)

def plot(data,phase,field,center=False,shape=(400,400),scale=None):
    detalle=['test_acc', 'test_teacher/acc', 'test_loss', 'test_eval',
           'train_acc', 'train_teacher/acc', 'train_loss', 'train_eval','distillation', 'temp']
    
    field_dict={'acc':"Accuracy [%]", 'eval':"Perdida Cross Entropy", 'loss':"Perdida de Destilación"
        
    }
    
    if scale is None:
        scale='log' if field in ['loss','eval'] else 'linear'
    else:
        print("scale", scale)
    
    print("lasorra")
    #data['train_acc']-=data['test_acc']
    
    
    bar=alt.Chart(data).mark_point().encode(
        alt.X('temp:O', scale=alt.Scale(zero=False,base=10,type='log', ),title="Temperatura"),
        alt.Y('%s_%s'%(phase,field), 
              scale=alt.Scale(zero=False, type=scale), 
              title=field_dict[field]),
        shape=alt.Shape('distillation', legend=alt.Legend(title="Destilación")),
        color=alt.Color('student', legend=alt.Legend(title="Modelo")),
        size=alt.value(50),
        tooltip=detalle
    ).interactive()
    
    if field == 'acc':
        accs = {'Model':['MobileNet','ResNet18','ResNet101'],
                      'ce_train':[95.73,98.15,98.52],
                      'ce_test':[87.8,90.58,90.68]}
        if center:
            d=dict(list(zip(accs['Model'],accs['ce_%s'%phase])))
            data['%s_acc'%phase]-=[d[i] for i in data['student']]        
        else:
            df=pd.DataFrame(accs)
            aggregates = alt.Chart(df).mark_rule(opacity=0.5).encode(
                        y='ce_%s:Q'%phase,
                        color='Model:N',
                        size=alt.value(2))
            
            return (aggregates+bar).properties(width=shape[0],height=shape[1]) 
        
    return bar.properties(width=shape[0],height=shape[1])
    
plot(data,phase='train',field="loss",center=True,scale="log")

scale log
lasorra


alt.Chart(...)

## Destilacion con features

In [263]:
import altair as alt
from lib.utils.viz import Feat
def acc_plot(data,field="acc",center=False,shape=(400,400),scale=None):
    detalle=['test_acc', 'test_teacher/acc', 'test_loss', 'test_eval',
           'train_acc', 'train_teacher/acc', 'train_loss', 'train_eval','distillation','last_layer','layer']
    
    field_dict={'acc':"Accuracy [%]", 'eval':"Perdida Cross Entropy", 'loss':"Perdida de Destilación"
        
    }
    
    data=data[data['distillation']!='nst_gauss']
    
    if scale is None:
        scale='log' if field in ['loss','eval'] else 'linear'
    else:
        print("scale", scale)
    
    print("lasorra")
    #data['train_acc']-=data['test_acc']
    
    
    bar=alt.Chart(data).mark_point().encode(
        alt.X('train_acc', scale=alt.Scale(zero=False,base=10,type=scale, ),title="Accuracy en entrenamiento"),
        alt.Y('test_acc', scale=alt.Scale(zero=False,base=10,type=scale, ),title="Accuracy en test"),
        shape=alt.Shape('distillation', legend=alt.Legend(title="Destilación")),
        color=alt.Color('student', legend=alt.Legend(title="Modelo")),
        fill=alt.Fill('last_layer', legend=alt.Legend(title="Destilacion logits")),
        size=alt.value(50),
        tooltip=detalle
        ).interactive()
    
    accs = {'Model':['MobileNet','ResNet18','ResNet101'],
                  'ce_train':[95.73,98.15,98.52],
                  'ce_test':[87.8,90.58,90.68]}
    if center:
        for phase in ["test","train"]:
            d=dict(list(zip(accs['Model'],accs['ce_%s'%phase])))
            data['%s_acc'%phase]-=[d[i] for i in data['student']]        
    else:
        
        df=pd.DataFrame(accs)

        aggregatey = alt.Chart(df).mark_rule(opacity=0.5).encode(
                    y='ce_test',
                    color='Model:N',
                    size=alt.value(2))

        aggregatex = alt.Chart(df).mark_rule(opacity=0.5).encode(
                    x='ce_train',
                    color='Model:N',
                    size=alt.value(2))

        return (aggregatey+aggregatex+bar).properties(width=shape[0],height=shape[1]) 
    return bar.properties(width=shape[0],height=shape[1])

folder="./Cifar10/ResNet101/exp7/students"

data=Feat.load_data(folder)
data["last_layer"]="KD"
acc_plot(data,scale="linear")

scale linear
lasorra


alt.LayerChart(...)

In [112]:
Feat.plot(data,phase="train",field="acc")

alt.LayerChart(...)

In [85]:
Feat.load_and_plot(folder=folder,phase="train",field="loss")

alt.Chart(...)

In [ ]:
plot(data,"test","acc",scale="symlog",center=True)

In [113]:
Feat.omniplot(folder="./Cifar10/ResNet101/exp6/students",scale="log")

alt.RepeatChart(...)

In [ ]:
accs = {'Model':['MobileNet','ResNet18','ResNet101'],
                      'ce_train':[95.73,98.15,98.52],
                      'ce_test':[87.8,90.58,90.68]}

d=dict(list(zip(accs['Model'],accs['ce_%s'%"train"])))



In [81]:
data

,Unnamed: 0,Unnamed: 0.1,test_acc,test_teacher/acc,test_loss,test_eval,train_acc,train_teacher/acc,train_loss,train_eval,...,train_batch_size,test_batch_size,student,teacher,distillation,last_layer,layer,student_layer,teacher_layer,student last_layer
0,0,0,10.06,90.68,4.409160e+00,345.872999,10.000,9.998,3.321249e-04,12.940195,...,128,100,ResNet18,ResNet101,PKT,KD_CE,2,1.0,1.0,"ResNet18,KD_CE"
1,1,1,8.33,90.68,1.710562e+00,141.544908,10.000,10.000,3.177041e-04,12.943537,...,128,100,ResNet18,ResNet101,PKT,KD_CE,1,1.0,1.0,"ResNet18,KD_CE"
2,2,2,9.92,90.68,4.188128e+00,349.988649,10.000,10.000,2.505570e-04,12.969949,...,128,100,ResNet18,ResNet101,PKT,KD_CE,3,1.0,1.0,"ResNet18,KD_CE"
3,3,3,8.12,90.68,8.659277e-01,79.545480,10.000,10.004,4.973628e-04,12.939835,...,128,100,ResNet18,ResNet101,PKT,KD_CE,0,1.0,1.0,"ResNet18,KD_CE"
4,4,4,8.25,90.68,2.499623e+00,216.374351,10.000,10.000,3.503129e-04,12.978364,...,64,64,ResNet18,ResNet101,nst_linear,KD_CE,2,1.0,1.0,"ResNet18,KD_CE"
5,5,5,10.21,90.68,6.173399e-01,43.718721,10.000,10.000,4.608291e-03,12.970457,...,64,64,ResNet18,ResNet101,nst_linear,KD_CE,1,1.0,1.0,"ResNet18,KD_CE"
6,6,6,10.15,90.68,5.360548e+00,457.047252,10.000,10.000,1.428188e-04,12.971247,...,64,64,ResNet18,ResNet101,nst_linear,KD_CE,3,1.0,1.0,"ResNet18,KD_CE"
7,7,7,7.90,90.68,4.060999e+00,284.508376,10.000,10.002,1.566831e-02,12.969504,...,64,64,ResNet18,ResNet101,nst_linear,KD_CE,0,1.0,1.0,"ResNet18,KD_CE"
8,8,8,10.95,90.68,1.967298e-01,14.721685,10.000,10.000,2.320621e-03,12.965794,...,64,64,ResNet18,ResNet101,nst_poly,KD_CE,2,1.0,1.0,"ResNet18,KD_CE"
9,9,9,10.02,90.68,6.682788e+00,500.147658,10.000,10.000,9.263210e-02,12.967367,...,64,64,ResNet18,ResNet101,nst_poly,KD_CE,1,1.0,1.0,"ResNet18,KD_CE"


# Dataset Merging

In [162]:
import pandas as pd
def load_KD_data(folder="./Cifar10/ResNet101/exp1/students",center=True):
    data = pd.read_csv(folder+"/summary.csv") 
    data = data[data["student"] != 'EfficientNetB0']
    accs = {'Model':['MobileNet','ResNet18','ResNet101'],
                      'ce_train':[95.73,98.15,98.52],
                      'ce_test':[87.8,90.58,90.68]}
    if center:
        phase="test"
        d=dict(list(zip(accs['Model'],accs['ce_%s'%phase])))
        data['%s_acc'%phase]-=[d[i] for i in data['student']] 
            
    if "temp" not in data.columns:
        d_arr=[d.split(",")[0] for d in data["distillation"]]
        t_arr=[float(d.split(",T-")[1]) for d in data["distillation"]]
        data["distillation"]=d_arr
        data["temp"]=t_arr
        
    
    return data#[data["distillation"]!="KD_CE"]


def merge_KD(center=True):
    drop_cols="test_teacher/acc train_teacher/acc lr	epochs	train_batch_size teacher	test_batch_size resume epoch".split()


    d1=load_KD_data(folder="./Cifar10/ResNet101/exp1/students",center=center).drop(["test_teacher/acc","train_teacher/acc"],axis=1)
    d2=load_KD_data(folder="./Cifar10/ResNet101/exp2/students",center=center).drop(drop_cols,axis=1)
    d3=load_KD_data(folder="./Cifar10/ResNet101/exp3/students",center=center).drop(drop_cols,axis=1)
    db=d3.merge(d2,on=['student','distillation','temp'],suffixes=('_R01','_R1')).drop(['Unnamed: 0_R01','Unnamed: 0_R1'],axis=1)
    db=db.merge(d1,on=['student','distillation','temp']).drop(['epoch','Unnamed: 0'],axis=1)
    
    return db


def plot_ruido(center=True):
    db=merge_KD(center=True)
    db["mix"]=[", ".join([r['student'],r['distillation'] ])  for i,r in db.iterrows()]

    detalle=['test_acc' + s for s in ["","_R1",'_R01']]
    scale=alt.Scale(zero=False,base=10,type='symlog',domain=(-3,3) if center else (85,95))

    chart=alt.Chart(db).mark_point().encode(
        alt.Y("test_acc", type='quantitative',title="Accuracy dataset natural",scale=scale),#,scale=alt.Scale(zero=False,base=10,type='symlog')),
        alt.X("test_acc_R01", type='quantitative',title="Accuracy dataset con ruido",scale=scale),#,scale=alt.Scale(zero=False,base=10,type='symlog')),
        color=alt.Color('temp', scale=alt.Scale(zero=False,base=10,type='log', ),title="Temperatura"),
        fill=alt.Color('mix',title="Modelo, destilación"),
        shape=alt.Color('mix',title="Modelo, destilación"),
        size=alt.value(80)
    ).properties(
        width=400,
        height=400
    ).interactive()#.repeat(row=detalle,column=detalle)
    return chart

alt.Chart(...)

# features


In [321]:
import pandas as pd

def load_Feat_data(folder="./Cifar10/ResNet101/exp7/students",center=True):
    data = pd.read_csv(folder+"/summary.csv") 
    data = data[data["student"] != 'EfficientNetB0']
    accs = {'Model':['MobileNet','ResNet18','ResNet101'],
                      'ce_train':[95.73,98.15,98.52],
                      'ce_test':[87.8,90.58,90.68]}
    if center:
        phase="test"
        d=dict(list(zip(accs['Model'],accs['ce_%s'%phase])))
        data['%s_acc'%phase]-=[d[i] for i in data['student']] 
        
        
    
    
    return data#[data["last_layer"]!="KD_CE"]


def h_merge_Feat(center=True):
    drop_cols="test_teacher/acc train_teacher/acc lr	epochs	train_batch_size teacher pre	test_batch_size resume epoch last_layer  teacher_layer student_layer".split()
    merge_on='student distillation layer'.split()

    d1=load_Feat_data(folder="./Cifar10/ResNet101/exp7/students",center=center)
    d1=d1[d1["last_layer"]!="KD_CE"]
    d1=d1[d1["distillation"]!="nst_gauss"]
    #return d1.sort_values(by=["distillation","layer"])
    #d1.drop(drop_cols+['lambda'],axis=1)
    #print(d1.columns)
    d2=load_Feat_data(folder="./Cifar10/ResNet101/exp4/students",center=center).drop(drop_cols,axis=1)
    #print(d2.columns)
    d3=load_Feat_data(folder="./Cifar10/ResNet101/exp5/students",center=center).drop(drop_cols+['student last_layer'],axis=1)
    #print(d3)
    db=d3.merge(d2,on=merge_on,suffixes=('_R1','_R01')).drop(['Unnamed: 0_R01','Unnamed: 0_R1'],axis=1)
    #print(db['student'])
    db=db.merge(d1,on=merge_on)#.drop(['Unnamed: 0','Unnamed: 0.1_x','Unnamed: 0.1_y'],axis=1)
    
    return db

#load_Feat_data(folder="./Cifar10/ResNet101/exp5/students",center=True).drop(drop_cols,axis=1)
h_merge_Feat()

,Unnamed: 0.1,test_acc_R1,test_loss_R1,test_eval_R1,train_acc_R1,train_loss_R1,train_eval_R1,student,distillation,layer,...,test_acc,test_batch_size,test_eval,test_loss,test_teacher/acc,train_acc,train_batch_size,train_eval,train_loss,train_teacher/acc
0,0,-80.52,4.409160e+00,345.872999,10.000,3.321249e-04,12.940195,ResNet18,PKT,2,...,0.98,100,0.422881,2.754891e-03,90.68,97.356,128,0.088259,1.268059e-03,98.244
1,1,-82.25,1.710562e+00,141.544908,10.000,3.177041e-04,12.943537,ResNet18,PKT,1,...,0.74,100,0.443347,2.863789e-03,90.68,97.136,128,0.091023,1.294471e-03,98.176
2,2,-80.66,4.188128e+00,349.988649,10.000,2.505570e-04,12.969949,ResNet18,PKT,3,...,1.35,100,0.425011,2.784124e-03,90.68,97.450,128,0.080064,1.206720e-03,98.216
3,3,-82.46,8.659277e-01,79.545480,10.000,4.973628e-04,12.939835,ResNet18,PKT,0,...,1.11,100,0.446707,2.633350e-03,90.68,97.252,128,0.089405,1.276668e-03,98.234
4,4,-82.33,2.499623e+00,216.374351,10.000,3.503129e-04,12.978364,ResNet18,nst_linear,2,...,0.84,64,0.466651,4.891452e-03,90.68,96.904,64,0.103367,2.856537e-03,98.250
5,5,-80.37,6.173399e-01,43.718721,10.000,4.608291e-03,12.970457,ResNet18,nst_linear,1,...,-0.58,64,0.544904,9.014511e-03,90.68,94.562,64,0.199358,6.605168e-03,98.322
6,6,-80.43,5.360548e+00,457.047252,10.000,1.428188e-04,12.971247,ResNet18,nst_linear,3,...,0.71,64,0.440117,2.905455e-03,90.68,97.072,64,0.096831,1.421885e-03,98.234
7,7,-82.68,4.060999e+00,284.508376,10.000,1.566831e-02,12.969504,ResNet18,nst_linear,0,...,-1.73,64,0.630595,3.399673e-02,90.68,94.720,64,0.198311,2.348734e-02,98.266
8,8,-79.63,1.967298e-01,14.721685,10.000,2.320621e-03,12.965794,ResNet18,nst_poly,2,...,0.43,64,0.477990,8.389234e-03,90.68,96.368,64,0.128135,5.398843e-03,98.328
9,9,-80.56,6.682788e+00,500.147658,10.000,9.263210e-02,12.967367,ResNet18,nst_poly,1,...,-1.78,64,0.639786,1.022391e-01,90.68,94.788,64,0.191516,9.165701e-02,98.204


In [340]:


def plot_feat_ruido(center=True):
    db=h_merge_Feat(center=center)
    #db["mix"]=[", ".join([r['student'],r['distillation'] ])  for i,r in db.iterrows()]

    detalle=['test_acc' + s for s in ["","_R1",'_R01']]
    scale=alt.Scale(zero=False,base=10,type='symlog',domain=(-3,3) if center else (85,95))

    chart=alt.Chart(db).mark_point().encode(
        alt.Y("test_acc", type='quantitative',title="Accuracy dataset natural",scale=scale),#,scale=alt.Scale(zero=False,base=10,type='symlog')),
        alt.X("test_acc_R01", type='quantitative',title="Accuracy dataset con ruido",scale=scale),#,scale=alt.Scale(zero=False,base=10,type='symlog')),
        color=alt.Color('layer:O'),#, scale=alt.Scale(zero=False,base=10,type='log', ),title="Temperatura"),
        shape=alt.Color('distillation',title="Destilacion"),
        fill=alt.Color('student',title="Modelo"),
        size=alt.value(80)
    ).properties(
        width=400,
        height=400
    ).interactive()#.repeat(row=detalle,column=detalle)
    return chart

plot_feat_ruido(center=False)

alt.Chart(...)

In [265]:

!ls Cifar10/ResNet101/students/

MobileNet-KD   MobileNet-KDCE ResNet18-KD    ResNet18-KDCE


In [317]:
folders="MobileNet-KD   MobileNet-KDCE ResNet18-KD    ResNet18-KDCE".split()
pds=[pd.read_csv("Cifar10/ResNet101/students/%s/summary.csv"%s) for s in folders ]
pds=[p[p['lambda']!=	0.00001].drop(["lambda"],axis=1) if "lambda" in p.columns else p for p in pds]
pds=[p[p['distillation']!=	"nst_gauss"] for p in pds]


db=pd.concat(pds).drop("Unnamed: 0",axis=1)
db['last_layer']=["KD_CE" if r in ["KD_CE","KD-CE"] else "KD" for r in db['last_layer']]
db['layer']=[r['layer']-2 if r['student']=='ResNet18' else r['layer'] for i,r in db.iterrows()]

db.to_csv("./Cifar10/ResNet101/exp7/students/summary.csv")

/Users/jpcosec/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


,distillation,epoch,epochs,last_layer,layer,lr,pre,resume,student,student_layer,...,test_acc,test_batch_size,test_eval,test_loss,test_teacher/acc,train_acc,train_batch_size,train_eval,train_loss,train_teacher/acc
12,nst_poly,61,100,KD,3,0.01,50,False,MobileNet,55.0,...,87.96,64,0.692480,0.008205,90.68,91.414,64,0.378874,0.006234,98.178
13,nst_poly,92,100,KD,1,0.01,50,False,MobileNet,15.0,...,89.44,100,0.607345,0.025752,90.68,95.602,128,0.165215,0.013891,98.210
14,nst_poly,96,100,KD,0,0.01,50,False,MobileNet,6.0,...,89.34,100,0.666440,0.273567,90.68,94.854,128,0.200497,0.161489,98.226
15,nst_poly,94,100,KD,2,0.01,50,False,MobileNet,26.0,...,89.13,100,0.662776,0.012961,90.68,94.378,128,0.228835,0.007253,98.278
12,nst_poly,89,100,KD,3,0.01,50,False,MobileNet,55.0,...,89.22,64,0.409805,13.395901,90.68,96.980,64,0.084843,2.732834,98.294
12,nst_poly,88,100,KD,2,0.10,50,False,ResNet18,NaN,...,91.01,64,0.477990,0.008389,90.68,96.368,64,0.128135,0.005399,98.328
13,nst_poly,89,100,KD,1,0.10,50,False,ResNet18,NaN,...,88.80,64,0.639786,0.102239,90.68,94.788,64,0.191516,0.091657,98.204
14,nst_poly,87,100,KD,3,0.10,50,False,ResNet18,NaN,...,91.72,64,0.415898,0.002706,90.68,97.346,64,0.087293,0.001301,98.276
15,nst_poly,99,100,KD,0,0.10,50,False,ResNet18,NaN,...,78.65,100,1.278612,0.950857,90.68,78.274,128,1.085052,0.554614,98.248


In [48]:
import pandas as pd
def best_n(folder="./Cifar10/ResNet101/exp7/students",n=5, field=["test_acc"],return_tabular=True,
          info=["student","distillation","last_layer","test_acc","layer",'test_acc_deltaS','test_acc_deltaT']):
    data = pd.read_csv(folder+"/summary.csv") 
    data = data[data["student"] != 'EfficientNetB0']
    accs = {'Model':['MobileNet','ResNet18','ResNet101'],
                      'ce_train':[95.73,98.15,98.52],
                      'ce_test':[87.8,90.58,90.68]}
    for phase in ["test","train"]:
        d=dict(list(zip(accs['Model'],accs['ce_%s'%phase])))
        data['%s_acc_deltaS'%phase]=data['%s_acc'%phase]-[d[i] for i in data['student']] 
        data['%s_acc_deltaT'%phase]=data['%s_acc'%phase]-[d['ResNet101']] 
    data=data.sort_values(by=field,ascending=False)
    
    
    if return_tabular:
        
        return data.loc[:,data.columns.isin(info)].head(n).to_latex(index=False).split("\n")[4:4+n]
    return data.loc[:,data.columns.isin(info)].head(n)

def get_table(folder="./Cifar10/ResNet101/exp7/students", delta=False):

    ini="""
    \\begin{table}[]
    \\caption{ wea \\label{tab:wea}}
    \\centering
    \\begin{tabular}{|l|l|l|l|r|r|r|}
    \hline
    \\rowcolor[HTML]{9B9B9B} 
    Dest. features & Dest. logits & Bloque & Estudiante & Accuracy & $\\Delta$ estudiante & $\\Delta$ tutor \\\\ \\hline
    """     

    fin="""

    \end{tabular}
    \end{table}
    """
    if delta:
        table=ini+"\n".join([i+" \hline" for i in best_n(field="test_acc_deltaS")])+fin
    else:
        table=ini+"\n".join([i+" \hline" for i in best_n(field="test_acc")])+fin
        
    print(table)
    
    
get_table(delta=True)


    \begin{table}[]
    \caption{ wea \label{tab:wea}}
    \centering
    \begin{tabular}{|l|l|l|l|r|r|r|}
    \hline
    \rowcolor[HTML]{9B9B9B} 
    Dest. features & Dest. logits & Bloque & Estudiante & Accuracy & $\Delta$ estudiante & $\Delta$ tutor \\ \hline
      nst\_linear &         KD &      3 &  MobileNet &     90.66 &             2.86 &            -0.02 \\ \hline
  nst\_linear &      KD\_CE &      1 &  MobileNet &     90.65 &             2.85 &            -0.03 \\ \hline
     att\_max &         KD &      3 &  MobileNet &     90.55 &             2.75 &            -0.13 \\ \hline
         PKT &      KD\_CE &      0 &  MobileNet &     90.51 &             2.71 &            -0.17 \\ \hline
  nst\_linear &         KD &      2 &  MobileNet &     90.42 &             2.62 &            -0.26 \\ \hline

    \end{tabular}
    \end{table}
    


In [57]:
import pandas as pd
def best_n(folder="./Cifar10/ResNet101/exp1/students",n=5, field=["test_acc"],return_tabular=True,
          info=["student","distillation","test_acc","temp",'test_acc_deltaS','test_acc_deltaT']):
    data = pd.read_csv(folder+"/summary.csv") 
    data = data[data["student"] != 'EfficientNetB0']
    accs = {'Model':['MobileNet','ResNet18','ResNet101'],
                      'ce_train':[95.73,98.15,98.52],
                      'ce_test':[87.8,90.58,90.68]}
    for phase in ["test","train"]:
        d=dict(list(zip(accs['Model'],accs['ce_%s'%phase])))
        data['%s_acc_deltaS'%phase]=data['%s_acc'%phase]-[d[i] for i in data['student']] 
        data['%s_acc_deltaT'%phase]=data['%s_acc'%phase]-[d['ResNet101']] 
    data=data.sort_values(by=field,ascending=False)
    
    cols = data.columns.tolist()
    cols = ['Unnamed: 0','distillation', 'temp','student',  'test_acc', 'test_teacher/acc', 'test_loss',
            'test_eval', 'train_acc', 'train_teacher/acc', 'train_loss', 'train_eval', 'epoch', 'test_acc_deltaS', 
            'test_acc_deltaT', 'train_acc_deltaS', 'train_acc_deltaT']
    data=data[cols]
    if return_tabular:
        
        return data.loc[:,data.columns.isin(info)].head(n).to_latex(index=False).split("\n")[4:4+n]
    return data.loc[:,data.columns.isin(info)].head(n)

def get_table(folder="./Cifar10/ResNet101/exp7/students", delta=False):

    ini="""
    \\begin{table}[]
    \\caption{ wea \\label{tab:wea}}
    \\centering
    \\begin{tabular}{|l|l|l|l|r|r|r|}
    \hline
    \\rowcolor[HTML]{9B9B9B} 
    Destilacion & temperatura & Estudiante & Accuracy & $\\Delta$ estudiante & $\\Delta$ tutor \\\\ \\hline
    """     

    fin="""

    \end{tabular}
    \end{table}
    """
    if delta:
        table=ini+"\n".join([i+" \hline" for i in best_n(field="test_acc_deltaS")])+fin
    else:
        table=ini+"\n".join([i+" \hline" for i in best_n(field="test_acc")])+fin
        

    print(table)
get_table(delta=True)


    \begin{table}[]
    \caption{ wea \label{tab:wea}}
    \centering
    \begin{tabular}{|l|l|l|l|r|r|r|}
    \hline
    \rowcolor[HTML]{9B9B9B} 
    Destilacion & temperatura & Estudiante & Accuracy & $\Delta$ estudiante & $\Delta$ tutor \\ \hline
              KD &  1000.0 &  MobileNet &     90.42 &             2.62 &            -0.26 \\ \hline
          KD &  1000.0 &   ResNet18 &     91.94 &             1.36 &             1.26 \\ \hline
          KD &    10.0 &  MobileNet &     88.83 &             1.03 &            -1.85 \\ \hline
          KD &     1.0 &   ResNet18 &     91.58 &             1.00 &             0.90 \\ \hline
          KD &    50.0 &   ResNet18 &     91.52 &             0.94 &             0.84 \\ \hline

    \end{tabular}
    \end{table}
    


In [52]:
cols = ['Unnamed: 0','distillation', 'temp','student',  'test_acc', 'test_teacher/acc', 'test_loss', 'test_eval', 'train_acc', 'train_teacher/acc', 'train_loss', 'train_eval', 'epoch', 'test_acc_deltaS', 'test_acc_deltaT', 'train_acc_deltaS', 'train_acc_deltaT']

NameError: name 'df' is not defined